In [12]:
import spacy
# nlp = medspacy.load(disable={"ner"})
nlp = spacy.load("en_core_web_sm",disable={"ner"})


# create our custome patterns
target_rules = [
    {'label': "MUTATION" , 'pattern':'brca 1'},
    {'label': "MUTATION" , 'pattern':'brca 2'},
    {'label': "MUTATION" , 'pattern':'brca1'},
    {'label': "MUTATION" , 'pattern':'brca2'},
    {'label': "MUTATION" , 'pattern':'brca 1/2'},
    {'label': "MUTATION" , 'pattern':'brca 1/brca 2'},
    {'label': "MUTATION" , 'pattern':'brca 1'},
    {'label': "MUTATION" , 'pattern':'kras g12c'},
    
]

In [13]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']

In [14]:
ruler = nlp.add_pipe("entity_ruler")# create new pipline step
ruler.add_patterns(target_rules)# add our custome patterh here


In [15]:
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'entity_ruler']

In [16]:
doc = nlp("Discharge kras g12c Date: 11/15/2008. no chd3 had temp brca1 reading of 102.6 degrees. brca1")
doc.ents

(kras g12c, brca1, brca1)

In [21]:
for i in doc.ents:
#     print(type(i),i, i.text,sep="|")
    print(i.text, i.label_)

kras g12c MUTATION
brca1 MUTATION
brca1 MUTATION


In [35]:
def extract_mutations(text):
    text = str(text)
    doc = nlp(text)
    mutations = doc.ents
    mutations = list(set([i.text for i in mutations]))
    return "\n".join(mutations)

x = "Discharge kras g12c Date: 11/15/2008. no chd3 had temp brca1 reading of 102.6 degrees. brca1"

extract_mutations(x)

'brca1\nkras g12c'

In [42]:
from spacy import displacy

displacy.serve(doc, style='ent')


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [05/Mar/2023 15:57:37] "GET / HTTP/1.1" 200 1420
127.0.0.1 - - [05/Mar/2023 15:57:37] "GET /favicon.ico HTTP/1.1" 200 1420


Shutting down server on port 5000.


In [36]:
import pandas as pd
import numpy as np

df = pd.read_feather("a.feather")
df

,NCTId,Inclusion Criteria
0,NCT00001350,\n\n\n\n\n\n\ninclusion criteria:\n\n - iii.i....
1,NCT00001378,\ninclusion criteria:\n\n population character...
2,NCT00001806,\n\n\ninclusion criteria:\n\n - \n\ninclusion ...
3,NCT00003736,\ninclusion criteria:\n\n \n\n\ndisease charac...
4,NCT00004565,\n\n\ninclusion criteria:\n\n - \n\ninclusion ...
...,...,...
1234,NCT05158621,\n\n\ninclusion criteria:\n\n advanced/metasta...
1235,NCT05162443,\n\n\ninclusion criteria:\n\n \n\ninclusion cr...
1236,NCT05162755,\n\n\ninclusion criteria:\n\n dose escalation ...
1237,NCT05163028,\n\n\ninclusion criteria:\n\n key \n\ninclusio...


In [37]:
%%time
df['muation'] = df['Inclusion Criteria'].apply(extract_mutations)

CPU times: user 38.2 s, sys: 83 ms, total: 38.2 s
Wall time: 38.4 s


In [38]:
df

,NCTId,Inclusion Criteria,muation
0,NCT00001350,\n\n\n\n\n\n\ninclusion criteria:\n\n - iii.i....,
1,NCT00001378,\ninclusion criteria:\n\n population character...,brca1\nbrca2
2,NCT00001806,\n\n\ninclusion criteria:\n\n - \n\ninclusion ...,
3,NCT00003736,\ninclusion criteria:\n\n \n\n\ndisease charac...,brca1\nbrca2
4,NCT00004565,\n\n\ninclusion criteria:\n\n - \n\ninclusion ...,brca1\nbrca2
...,...,...,...
1234,NCT05158621,\n\n\ninclusion criteria:\n\n advanced/metasta...,
1235,NCT05162443,\n\n\ninclusion criteria:\n\n \n\ninclusion cr...,
1236,NCT05162755,\n\n\ninclusion criteria:\n\n dose escalation ...,
1237,NCT05163028,\n\n\ninclusion criteria:\n\n key \n\ninclusio...,


In [40]:
df.muation.value_counts(dropna=False)

                                          837
brca1\nbrca2                              269
brca1                                      35
brca 1                                     21
brca 1/2                                   20
kras g12c                                  19
brca2                                      14
brca 1\nbrca 2                              8
brca1\nbrca 2                               4
brca 1\nbrca 1/2                            2
brca1\nbrca 2\nbrca2                        2
brca1\nbrca 1\nbrca2                        2
brca1\nbrca 1/2\nbrca2                      2
brca1\nbrca 1                               1
brca1\nbrca 1\nbrca 2\nbrca2                1
brca1\nbrca 1\nbrca 2\nbrca 1/2\nbrca2      1
brca1\nbrca 1/2                             1
Name: muation, dtype: int64

In [49]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern = [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}]
matcher.add("HelloWorld", [pattern])

doc = nlp("Hello, world! Hello world!")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
#     span = doc[start:end]  # The matched span
#     print(span, start, end)
    print(match_id, string_idlo, world, start, end, span.text)


15578876784678163569 HelloWorld 0 3 Hello, world
